# 🧪 Test des Modules NLP - Offres Réelles

**Objectif** : Tester les 3 modules NLP sur un échantillon de 10 offres depuis la BDD

**Modules testés** :
1. ✅ `text_cleaner.py` - Nettoyage et lemmatisation
2. ✅ `skill_extractor.py` - Extraction compétences (tech + soft)
3. ✅ `info_extractor.py` - Extraction infos structurées (salaire, XP, formation, remote)

## 📦 Imports et Configuration

In [1]:
# Imports standards
import os
import sys
import psycopg2
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Ajouter le chemin des modules NLP
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'modules'))

# Modules NLP
from text_cleaner import TextCleaner
from skill_extractor import SkillExtractor
from info_extractor import InfoExtractor

# Config
load_dotenv()
DATABASE_URL = os.getenv('DATABASE_URL')

print("✅ Imports OK")
print(f"📍 Modules NLP chargés depuis : {os.path.join(os.path.dirname(os.getcwd()), 'modules')}")

✅ Imports OK
📍 Modules NLP chargés depuis : c:\Users\cyraptor\Documents\PROJECTS\Projet-ATLAS\NLP\modules


## 🔌 Connexion BDD et Chargement des Offres

In [2]:
# Connexion
conn = psycopg2.connect(DATABASE_URL)
print("✅ Connecté à PostgreSQL")

# Charger 10 offres aléatoires FILTRÉES SUR TOPICS TECH
query = """
SELECT 
    offer_id,
    title,
    description,
    company_name,
    contract_type,
    salary_min,
    salary_max,
    experience_years,
    topic_id,
    topic_label,
    topic_confidence
FROM fact_job_offers
WHERE description IS NOT NULL 
  AND LENGTH(description) > 200
  AND topic_label IN (
      'Ingénierie Cloud & Cybersécurité',
      'Data Analysis & Transformation Digitale',
      'Ingénierie R&D & Data Science',
      'Product Management & Développement Java',
      'Gestion de Projet & Développement'
  )
ORDER BY RANDOM()
LIMIT 10
"""

df = pd.read_sql(query, conn)
conn.close()

print(f"\n📊 {len(df)} offres TECH chargées")
print(f"\nAperçu :")
df[['offer_id', 'title', 'company_name', 'topic_label']].head()

✅ Connecté à PostgreSQL

📊 10 offres TECH chargées

Aperçu :


,offer_id,title,company_name,topic_label
0,674,Chef de projet électricité H/F,Work & You,Ingénierie Cloud & Cybersécurité
1,4423,DIRECTEUR COMMUNICATION & MARKETING DIGITAL - ...,Bras droit des dirigeants,Ingénierie Cloud & Cybersécurité
2,1198,Chef de Projets Data H/F,JEMS,Ingénierie Cloud & Cybersécurité
3,2038,Enquêteur / Enquêtrice terrain (H/F),BVA,Ingénierie R&D & Data Science
4,364,Chef de projet WMS (H/F),E-KENT,Ingénierie Cloud & Cybersécurité


## 🧹 Initialisation des Modules NLP

In [3]:
print("⏳ Initialisation des modules NLP...")

# Instancier les modules
cleaner = TextCleaner()
skill_extractor = SkillExtractor()
info_extractor = InfoExtractor()

print("✅ Modules NLP initialisés")
print(f"   - TextCleaner : modèle spaCy chargé")
print(f"   - SkillExtractor : {len(skill_extractor.all_tech_skills)} compétences tech + {len(skill_extractor.soft_skills)} soft skills")
print(f"   - InfoExtractor : extraction salaire, XP, formation, remote")

⏳ Initialisation des modules NLP...
✅ Modules NLP initialisés
   - TextCleaner : modèle spaCy chargé
   - SkillExtractor : 218 compétences tech + 76 soft skills
   - InfoExtractor : extraction salaire, XP, formation, remote


## 🔬 Fonction de Traitement NLP

In [4]:
def process_offer_nlp(offer_data):
    """
    Applique les 3 modules NLP sur une offre
    
    Returns:
        dict avec tous les résultats NLP
    """
    description = offer_data['description']
    
    # 1. NETTOYAGE
    cleaned = cleaner.clean_text(description)
    lemmas = cleaner.lemmatize(cleaned)
    
    # 2. EXTRACTION SKILLS
    skills = skill_extractor.extract_skills(description)
    category = skill_extractor.categorize_offer(description)
    
    # Calcul du profile_confidence (en pourcentage)
    # Score normalisé basé sur le nombre de skills détectées
    max_score = 10  # On considère 10+ skills comme 100%
    profile_confidence = min(100, int((category['profile_score'] / max_score) * 100))
    
    # 3. EXTRACTION INFOS
    info = info_extractor.extract_all(description)
    
    return {
        # Texte
        'cleaned_text': cleaned,
        'lemmas_count': len(lemmas),
        
        # Skills
        'skills_tech': skills['all_tech_skills'],
        'skills_soft': skills['soft_skills'],
        'skill_count_tech': skills['skill_count']['tech'],
        'skill_count_soft': skills['skill_count']['soft'],
        
        # Profil
        'profile_category': category['dominant_profile'],
        'profile_score': category['profile_score'],
        'profile_confidence': profile_confidence,  # EN POURCENTAGE
        'is_full_stack': category['is_full_stack'],
        
        # Infos structurées
        'salary_min': info['salary']['min'],
        'salary_max': info['salary']['max'],
        'experience_min': info['experience']['min'],
        'experience_max': info['experience']['max'],
        'experience_level': info['experience']['level'],
        'education_level': info['education']['level'],
        'education_type': info['education']['degree_type'],
        'contract_types': info['contract_types'],
        'remote_possible': info['remote']['remote_possible'],
        'remote_days': info['remote']['remote_days'],
        'remote_percentage': info['remote']['remote_percentage'],
    }

print("✅ Fonction de traitement définie")

✅ Fonction de traitement définie


## 🚀 Traitement des 10 Offres

In [5]:
print("="*80)
print("🚀 TRAITEMENT DES OFFRES")
print("="*80)
print()

results = []

for idx, row in df.iterrows():
    print(f"\n📌 Offre {idx + 1}/{len(df)} : {row['title'][:60]}...")
    
    # Traiter
    nlp_result = process_offer_nlp(row)
    
    # Combiner avec données originales
    result = {
        'offer_id': row['offer_id'],
        'title': row['title'],
        'company_name': row['company_name'],
        **nlp_result
    }
    
    results.append(result)
    
    # Affichage rapide
    print(f"   ✅ {result['skill_count_tech']} skills tech | {result['skill_count_soft']} soft skills")
    print(f"   🎯 Profil : {result['profile_category']} (confiance: {result['profile_confidence']}%)")

print(f"\n{'='*80}")
print(f"✅ {len(results)} offres traitées")
print(f"{'='*80}")

# Créer DataFrame des résultats
df_results = pd.DataFrame(results)

🚀 TRAITEMENT DES OFFRES


📌 Offre 1/10 : Chef de projet électricité H/F...
   ✅ 0 skills tech | 6 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 2/10 : DIRECTEUR COMMUNICATION & MARKETING DIGITAL - INDEPENDANT (H...
   ✅ 0 skills tech | 1 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 3/10 : Chef de Projets Data H/F...
   ✅ 3 skills tech | 8 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 4/10 : Enquêteur / Enquêtrice terrain (H/F)...
   ✅ 2 skills tech | 2 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 5/10 : Chef de projet WMS (H/F)...
   ✅ 2 skills tech | 7 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 6/10 : Candidature spontanée - Theodo Data & AI...
   ✅ 0 skills tech | 0 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 7/10 : Animateur de communauté (H/F)...
   ✅ 1 skills tech | 4 soft skills
   🎯 Profil : Généraliste (confiance: 0%)

📌 Offre 8/10 : Chargé(e) d'Études de Prix CVC (H/F).

## 📊 Résultats Détaillés par Offre

In [6]:
# Afficher résultats détaillés pour chaque offre
for idx, result in enumerate(results, 1):
    print("\n" + "="*80)
    print(f"📋 OFFRE #{idx} : {result['title']}")
    print("="*80)
    print(f"   Entreprise : {result['company_name']}")
    
    print(f"\n🧹 NETTOYAGE :")
    print(f"   Texte nettoyé : {len(result['cleaned_text'])} caractères")
    print(f"   Lemmes extraits : {result['lemmas_count']}")
    
    print(f"\n💻 COMPÉTENCES TECHNIQUES ({result['skill_count_tech']}) :")
    if result['skills_tech']:
        for i, skill in enumerate(result['skills_tech'][:15], 1):  # Top 15
            print(f"   {i:2}. {skill}")
        if len(result['skills_tech']) > 15:
            print(f"   ... et {len(result['skills_tech']) - 15} autres")
    else:
        print("   ℹ️  Aucune compétence technique détectée")
    
    print(f"\n🤝 SOFT SKILLS ({result['skill_count_soft']}) :")
    if result['skills_soft']:
        print(f"   {', '.join(result['skills_soft'][:10])}")
        if len(result['skills_soft']) > 10:
            print(f"   ... et {len(result['skills_soft']) - 10} autres")
    else:
        print("   ℹ️  Aucune soft skill détectée")
    
    print(f"\n🎯 CATÉGORISATION :")
    print(f"   Profil dominant : {result['profile_category']}")
    print(f"   Score : {result['profile_score']} compétences matchées")
    print(f"   Confiance : {result['profile_confidence']}%")
    if result['is_full_stack']:
        print(f"   ⚠️  Profil Full Stack détecté !")
    
    print(f"\n📋 INFORMATIONS STRUCTURÉES :")
    
    # Salaire
    if result['salary_min'] or result['salary_max']:
        print(f"   💰 Salaire : {result['salary_min']:,}€ - {result['salary_max']:,}€ /an")
    else:
        print(f"   💰 Salaire : Non spécifié")
    
    # Expérience
    if result['experience_min'] is not None:
        if result['experience_min'] == result['experience_max']:
            print(f"   📅 Expérience : {result['experience_min']} ans ({result['experience_level']})")
        else:
            print(f"   📅 Expérience : {result['experience_min']}-{result['experience_max']} ans ({result['experience_level']})")
    else:
        print(f"   📅 Expérience : Non spécifiée")
    
    # Formation
    if result['education_level']:
        print(f"   🎓 Formation : Bac+{result['education_level']} ({result['education_type']})")
    else:
        print(f"   🎓 Formation : Non spécifiée")
    
    # Contrat
    if result['contract_types']:
        print(f"   📝 Contrat : {', '.join(result['contract_types'])}")
    else:
        print(f"   📝 Contrat : Non spécifié")
    
    # Télétravail
    if result['remote_possible']:
        if result['remote_days']:
            print(f"   🏠 Télétravail : {result['remote_days']} jours/semaine ({result['remote_percentage']}%)")
        elif result['remote_percentage']:
            print(f"   🏠 Télétravail : {result['remote_percentage']}%")
        else:
            print(f"   🏠 Télétravail : Possible (détails non précisés)")
    else:
        print(f"   🏠 Télétravail : Non mentionné")


📋 OFFRE #1 : Chef de projet électricité H/F
   Entreprise : Work & You

🧹 NETTOYAGE :
   Texte nettoyé : 1686 caractères
   Lemmes extraits : 123

💻 COMPÉTENCES TECHNIQUES (0) :
   ℹ️  Aucune compétence technique détectée

🤝 SOFT SKILLS (6) :
   analyse, autonome, collaboration, dynamique, esprit d'équipe, rigoureux

🎯 CATÉGORISATION :
   Profil dominant : Généraliste
   Score : 0 compétences matchées
   Confiance : 0%

📋 INFORMATIONS STRUCTURÉES :
   💰 Salaire : Non spécifié
   📅 Expérience : 3-6 ans (Confirmé)
   🎓 Formation : Bac+5 (Master/Ingénieur)
   📝 Contrat : CDI
   🏠 Télétravail : Possible (détails non précisés)

📋 OFFRE #2 : DIRECTEUR COMMUNICATION & MARKETING DIGITAL - INDEPENDANT (H/F)
   Entreprise : Bras droit des dirigeants

🧹 NETTOYAGE :
   Texte nettoyé : 2430 caractères
   Lemmes extraits : 180

💻 COMPÉTENCES TECHNIQUES (0) :
   ℹ️  Aucune compétence technique détectée

🤝 SOFT SKILLS (1) :
   communication

🎯 CATÉGORISATION :
   Profil dominant : Généraliste
   Scor

## 📈 Statistiques Globales

In [7]:
print("\n" + "="*80)
print("📈 STATISTIQUES GLOBALES")
print("="*80)

print(f"\n🎯 Profils détectés :")
profile_counts = df_results['profile_category'].value_counts()
for profile, count in profile_counts.items():
    print(f"   • {profile:40} : {count} offre(s)")

print(f"\n💻 Compétences techniques :")
print(f"   Moyenne par offre : {df_results['skill_count_tech'].mean():.1f}")
print(f"   Médiane : {df_results['skill_count_tech'].median():.0f}")
print(f"   Min / Max : {df_results['skill_count_tech'].min()} / {df_results['skill_count_tech'].max()}")

print(f"\n🤝 Soft skills :")
print(f"   Moyenne par offre : {df_results['skill_count_soft'].mean():.1f}")
print(f"   Médiane : {df_results['skill_count_soft'].median():.0f}")
print(f"   Min / Max : {df_results['skill_count_soft'].min()} / {df_results['skill_count_soft'].max()}")

print(f"\n🎯 Confiance profil :")
print(f"   Moyenne : {df_results['profile_confidence'].mean():.1f}%")
print(f"   Médiane : {df_results['profile_confidence'].median():.0f}%")
print(f"   Min / Max : {df_results['profile_confidence'].min()}% / {df_results['profile_confidence'].max()}%")

print(f"\n📊 Informations extraites :")
salary_detected = df_results['salary_min'].notna().sum()
exp_detected = df_results['experience_min'].notna().sum()
edu_detected = df_results['education_level'].notna().sum()
contract_detected = df_results['contract_types'].apply(lambda x: len(x) > 0).sum()
remote_detected = df_results['remote_possible'].sum()

print(f"   Salaire détecté : {salary_detected}/{len(df_results)} offres ({salary_detected/len(df_results)*100:.0f}%)")
print(f"   Expérience détectée : {exp_detected}/{len(df_results)} offres ({exp_detected/len(df_results)*100:.0f}%)")
print(f"   Formation détectée : {edu_detected}/{len(df_results)} offres ({edu_detected/len(df_results)*100:.0f}%)")
print(f"   Contrat détecté : {contract_detected}/{len(df_results)} offres ({contract_detected/len(df_results)*100:.0f}%)")
print(f"   Télétravail détecté : {remote_detected}/{len(df_results)} offres ({remote_detected/len(df_results)*100:.0f}%)")


📈 STATISTIQUES GLOBALES

🎯 Profils détectés :
   • Généraliste                              : 10 offre(s)

💻 Compétences techniques :
   Moyenne par offre : 0.8
   Médiane : 0
   Min / Max : 0 / 3

🤝 Soft skills :
   Moyenne par offre : 3.5
   Médiane : 3
   Min / Max : 0 / 8

🎯 Confiance profil :
   Moyenne : 0.0%
   Médiane : 0%
   Min / Max : 0% / 0%

📊 Informations extraites :
   Salaire détecté : 0/10 offres (0%)
   Expérience détectée : 2/10 offres (20%)
   Formation détectée : 5/10 offres (50%)
   Contrat détecté : 5/10 offres (50%)
   Télétravail détecté : 1/10 offres (10%)


## 🔝 Top 20 Compétences Techniques

In [8]:
# Agréger toutes les skills tech
all_tech_skills = []
for skills_list in df_results['skills_tech']:
    all_tech_skills.extend(skills_list)

# Compter
from collections import Counter
skill_counts = Counter(all_tech_skills).most_common(20)

print("\n" + "="*80)
print("🔝 TOP 20 COMPÉTENCES TECHNIQUES")
print("="*80)
print()

for rank, (skill, count) in enumerate(skill_counts, 1):
    pct = (count / len(df_results)) * 100
    print(f"   {rank:2}. {skill:30} : {count} offres ({pct:.0f}%)")


🔝 TOP 20 COMPÉTENCES TECHNIQUES

    1. agile                          : 2 offres (20%)
    2. jira                           : 1 offres (10%)
    3. safe                           : 1 offres (10%)
    4. sécurité                       : 1 offres (10%)
    5. teams                          : 1 offres (10%)
    6. go                             : 1 offres (10%)
    7. rgpd                           : 1 offres (10%)


## 💾 Export des Résultats (Optionnel)

In [9]:
# Exporter en CSV pour analyse
output_file = "test_nlp_results_10offres.csv"

# Préparer colonnes pour CSV (simplifier les listes)
df_export = df_results.copy()
df_export['skills_tech_str'] = df_export['skills_tech'].apply(lambda x: ', '.join(x[:10]))
df_export['skills_soft_str'] = df_export['skills_soft'].apply(lambda x: ', '.join(x[:10]))
df_export['contract_types_str'] = df_export['contract_types'].apply(lambda x: ', '.join(x))

# Colonnes à exporter
cols_to_export = [
    'offer_id', 'title', 'company_name',
    'lemmas_count',
    'skill_count_tech', 'skill_count_soft',
    'skills_tech_str', 'skills_soft_str',
    'profile_category', 'profile_score', 'profile_confidence',
    'salary_min', 'salary_max',
    'experience_min', 'experience_max', 'experience_level',
    'education_level', 'education_type',
    'contract_types_str',
    'remote_possible', 'remote_days', 'remote_percentage'
]

df_export[cols_to_export].to_csv(output_file, index=False, encoding='utf-8')

print(f"\n✅ Résultats exportés : {output_file}")
print(f"   {len(df_export)} offres traitées")


✅ Résultats exportés : test_nlp_results_10offres.csv
   10 offres traitées
